In [19]:
import pandas as pd
import numpy as np
import time
import sklearn
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import numpy as np
import scipy.cluster.hierarchy as hclust
import pylab as pl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, cut_tree
import kmedoids

In [20]:
df_log = pd.read_csv("C:/Users/djcsr/Documents/Semester 9/CGS4144 Bioinformatics/Analysis/data/SRP073813/SRP073813_log.csv")
df_meta = pd.read_csv("C:/Users/djcsr/Documents/Semester 9/CGS4144 Bioinformatics/Analysis/data/SRP073813/SRP073813_meta.csv")

In [21]:
df_log.set_index("HUGO", inplace = True)
df_meta.set_index("ID", inplace = True) #Setting the index for data makes the enrichment analysis easier
df_meta = df_meta['refinebio_subject'] #removes unnecessary data 

In [22]:
df_meta = df_meta.to_frame()

In [23]:
df_log =round(df_log) 
df_log = df_log.T #swaps columns and indexes which allows the deseq to run 
df_log = df_log.join(df_meta)

In [24]:
df_log = df_log.loc[:, ~df_log.columns.duplicated()] #removes duplicate columns prevent deseq

In [25]:
#Transfomring the data to contain only data collected from a certain part of the brain in our patients 
df_log = df_log[df_log["refinebio_subject"].str.contains('nacc')]
df_meta = df_meta[df_meta["refinebio_subject"].str.contains('nacc')] 

In [26]:
df_log.set_index("refinebio_subject", inplace = True)

In [27]:
df_log = df_log.T

In [28]:
df_log["variance"] = np.var(df_log.values, axis=1)
df = df_log.sort_values("variance", ascending=False)
dfTen = df.head(10)
dfHun = df.head(100)
dfThous = df.head(1000)
dfTenThous = df.head(10000)
dfFiveThous = df.head(5000)
df.drop("variance", axis =1, inplace = True)

In [29]:
# Above is code needed until we diverge on clustering method

In [30]:
dfTen.shape

(10, 115)

In [55]:
dissDfTen = euclidean_distances(dfTen)
dissDfHun = euclidean_distances(dfHun)
dissDfThous = euclidean_distances(dfThous)
dissDfTenThous = euclidean_distances(dfTenThous)
dissDfFiveThous = euclidean_distances(dfFiveThous)

In [57]:
# clustering using PAM on ten genes
tenPamk5 = kmedoids.pam(dissDfTen, medoids = 5, max_iter = 100, init = "first", random_state = None) 
tenPamk4 = kmedoids.pam(dissDfTen, medoids = 4, max_iter = 100, init = "first", random_state = None) 
tenPamk3 = kmedoids.pam(dissDfTen, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [58]:
tenPamk5, tenPamk4, tenPamk3

(KMedoidsResult(loss=43.96174259137469, labels=[0 2 2 2 4 3 2 2 1 2], medoids=[0 8 2 5 4], n_iter=3, n_swaps=2),
 KMedoidsResult(loss=58.418584351908294, labels=[0 2 2 2 1 3 2 2 3 2], medoids=[0 4 2 5], n_iter=3, n_swaps=2),
 KMedoidsResult(loss=75.28332167186747, labels=[0 2 2 2 0 1 2 2 1 2], medoids=[0 5 2], n_iter=2, n_swaps=1))

In [61]:
#clustering using PAM on one hundred genes
hunPamk5 = kmedoids.pam(dissDfHun, medoids = 5, max_iter = 100, init = "first", random_state = None) 
hunPamk4 = kmedoids.pam(dissDfHun, medoids = 4, max_iter = 100, init = "first", random_state = None) 
hunPamk3 = kmedoids.pam(dissDfHun, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [62]:
hunPamk5, hunPamk4, hunPamk3

(KMedoidsResult(loss=842.5143802762627, labels=[0 1 3 3 0 2 3 1 2 3 2 3 3 4 3 2 0 2 1 3 1 4 4 1 3 3 1 1 1 2 2 3 1 1 3 4 3
  4 1 2 3 3 3 3 2 0 1 2 3 1 0 1 3 2 2 2 1 3 3 3 2 2 1 1 2 0 0 0 1 3 3 2 1 2
  1 3 1 3 1 1 1 2 4 4 3 1 3 0 1 3 1 4 1 1 1 1 0 1 2 2], medoids=[67 94 71 24 83], n_iter=7, n_swaps=6),
 KMedoidsResult(loss=906.4576073600996, labels=[0 1 3 3 0 2 3 1 2 3 2 3 3 0 3 2 0 2 1 3 1 0 0 1 3 3 1 1 1 2 2 3 1 1 3 0 3
  0 2 1 3 3 3 3 2 0 1 2 3 1 0 1 3 2 2 2 1 3 3 3 2 2 1 1 2 0 2 0 1 3 3 2 1 2
  1 3 1 3 1 1 1 1 0 0 3 1 3 2 1 3 1 0 1 1 1 1 0 1 2 2], medoids=[83 94 98 24], n_iter=6, n_swaps=5),
 KMedoidsResult(loss=1003.0630397226475, labels=[0 1 2 2 0 1 2 2 1 2 1 2 2 1 2 1 0 1 1 2 1 1 1 1 2 2 1 1 1 0 1 2 2 2 2 1 2
  1 1 2 2 2 2 2 0 0 1 1 2 2 0 2 2 1 1 0 1 2 2 2 1 1 1 1 0 0 0 0 1 2 2 1 1 1
  2 2 2 2 2 1 1 2 0 1 2 1 2 0 1 2 1 1 1 2 1 1 0 1 1 0], medoids=[66 88 24], n_iter=4, n_swaps=3))

In [63]:
#clustering using PAM on one thousand genes
thousPamk5 = kmedoids.pam(dissDfThous, medoids = 5, max_iter = 100, init = "first", random_state = None) 
thousPamk4 = kmedoids.pam(dissDfThous, medoids = 4, max_iter = 100, init = "first", random_state = None) 
thousPamk3 = kmedoids.pam(dissDfThous, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [64]:
thousPamk5, thousPamk4, thousPamk3

(KMedoidsResult(loss=7875.221585347835, labels=[4 2 2 2 4 3 2 2 2 2 3 2 2 0 2 3 4 2 1 2 1 0 0 1 2 2 1 1 1 3 3 2 2 2 2 0 2
  0 1 2 2 2 2 2 3 4 2 3 2 2 4 2 2 3 3 3 1 2 2 2 2 3 1 1 3 4 4 4 2 2 2 3 1 2
  2 2 2 2 2 1 1 2 0 0 2 1 2 4 1 2 2 0 1 2 2 1 4 1 3 3 2 3 2 1 2 3 4 2 1 1 2
  2 1 2 3 2 3 2 2 0 2 2 1 1 2 2 2 2 2 3 2 1 3 1 1 2 1 3 0 3 2 1 2 2 2 4 2 1
  1 1 2 1 2 2 2 1 0 4 1 1 2 2 1 1 1 3 2 0 2 1 2 1 2 2 2 4 1 4 1 1 3 2 3 0 2
  2 1 3 1 1 3 2 2 4 0 2 2 2 0 2 2 1 2 3 2 2 2 0 2 2 4 1 0 2 2 2 3 1 3 4 1 3
  1 3 2 2 3 0 3 2 0 0 0 1 2 2 3 3 3 2 2 1 1 3 2 3 1 3 2 1 4 1 0 3 1 1 1 2 2
  0 3 2 1 2 2 1 1 1 3 4 2 2 1 2 2 1 4 2 2 3 1 2 2 2 1 3 1 3 0 4 2 1 1 4 1 3
  1 3 2 4 2 1 1 1 2 2 0 2 3 2 2 2 0 4 1 2 0 2 2 3 1 2 3 1 3 2 2 0 4 3 0 2 1
  2 3 1 1 1 3 1 3 1 2 1 3 3 0 0 1 0 4 0 4 1 0 1 3 1 3 2 1 0 4 3 1 2 1 1 3 1
  1 2 4 2 2 3 3 3 3 4 1 1 3 4 3 4 1 4 0 2 2 1 1 4 0 1 2 1 4 1 0 3 1 0 1 0 1
  2 1 1 3 3 3 1 2 1 0 1 3 0 1 1 2 1 1 1 1 3 4 1 4 2 3 3 0 0 2 1 2 1 1 4 1 1
  2 3 0 0 0 0 2 0 2 1 1 0 3 0 2 1 1 3 2 0 

In [67]:
#clustering using PAM on ten thousand genes
tenThousPamk5 = kmedoids.pam(dissDfTenThous, medoids = 5, max_iter = 100, init = "first", random_state = None) 
tenThousPamk4 = kmedoids.pam(dissDfTenThous, medoids = 4, max_iter = 100, init = "first", random_state = None) 
tenThousPamk3 = kmedoids.pam(dissDfTenThous, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [70]:
tenThousPamk5, tenThousPamk4, tenThousPamk3

(KMedoidsResult(loss=61658.200279695644, labels=[0 3 3 ... 0 0 0], medoids=[9913 9797 8388 2043 9957], n_iter=7, n_swaps=6),
 KMedoidsResult(loss=65743.55198596789, labels=[0 1 1 ... 0 0 0], medoids=[9913 8388 9957 9797], n_iter=6, n_swaps=5),
 KMedoidsResult(loss=70439.62158928667, labels=[0 1 1 ... 0 0 0], medoids=[9913 8388 9784], n_iter=6, n_swaps=5))

In [71]:
#clustering using PAM on five thousand genes
fiveThousPamk5 = kmedoids.pam(dissDfFiveThous, medoids = 5, max_iter = 100, init = "first", random_state = None) 
fiveThousPamk4 = kmedoids.pam(dissDfFiveThous, medoids = 4, max_iter = 100, init = "first", random_state = None) 
fiveThousPamk3 = kmedoids.pam(dissDfFiveThous, medoids = 3, max_iter = 100, init = "first", random_state = None) 

In [72]:
fiveThousPamk5, fiveThousPamk4, fiveThousPamk3

(KMedoidsResult(loss=36868.353132816446, labels=[0 1 1 ... 3 3 4], medoids=[4908 2043 4899 4463 4867], n_iter=11, n_swaps=10),
 KMedoidsResult(loss=38275.32180685905, labels=[0 1 1 ... 0 0 3], medoids=[4797 2043 4899 4934], n_iter=7, n_swaps=6),
 KMedoidsResult(loss=43032.85209672146, labels=[0 1 1 ... 0 0 2], medoids=[4984 4754 4934], n_iter=6, n_swaps=5))